### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221109_scenario_projections.ipynb")

2022/11/04 20:30:49 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/11/04 20:30:52 INFO|-------------------------|{} Retrieving vaccinations data
2022/11/04 20:30:54 INFO|-------------------------|{} Constructing vaccination projections
2022/11/04 20:30:56 INFO|-------------------------|{} Retrieving hospitalizations data
2022/11/04 20:30:59 INFO|----------------------|{} Prepping Model
2022/11/04 20:30:59 INFO|-----------------------|{} Prepping Model
2022/11/04 20:32:24 INFO|----------------------|{} Model prepped for fitting in 85.060838792 seconds.
2022/11/04 20:32:24 INFO|----------------------|{} Will fit 25 times
2022/11/04 20:33:43 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 79.34436637499999 seconds: {0: {'co': 0.8306366903708943}, 14: {'co': 0.6980458511135594}, 28: {'co': 0.5697135335788563}, 42: {'co': 0.0028157696761687873}, 56: {'co': 0.6814708839651918}, 70: {'co': 0.8912746324074166}}
2022/11/04 20:33:43 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/11/04 20

2022/11/04 20:39:34 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 11.830518583000071 seconds: {504: {'co': 0.7862680304298327}, 518: {'co': 0.780069311987493}, 532: {'co': 0.7569552785459293}, 546: {'co': 0.7093855296149169}, 560: {'co': 0.7432183462181008}, 574: {'co': 0.7376886963420669}}
2022/11/04 20:39:34 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/11/04 20:39:59 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 22.21647791700002 seconds: {532: {'co': 0.7562240331024491}, 546: {'co': 0.7114553688154189}, 560: {'co': 0.7394587228300054}, 574: {'co': 0.7494337123859183}, 588: {'co': 0.7863407479413587}, 602: {'co': 0.7653149034274409}}
2022/11/04 20:39:59 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/11/04 20:40:16 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 15.84270304200004 seconds: {5

### Create and run scenarios from Feb 2022 to present

In [14]:
multiprocess = 3

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': 4241, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for escape_mult in [0.8, 1, 1.2]:
    hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
           "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
    ie_adjust = [{"param": "immune_escape",
                   "from_attrs": {"variant": ["none", "wildtype", "alpha", "delta", "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["scrabble"]},
                   "mults": {"2020-01-01": escape_mult},
                   "desc": "Scrabble variants immune escape multiplier"}]
    lt5_vacc_adjust = [{"param": "immunity",
                        "attrs": {'age': '0-19', 'vacc': 'shot1'},
                        "mults": {"2020-01-01": 1,
                                  "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                  "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                  "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                  "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                  "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                  "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                  "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                  "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                  "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
    scenario_model_args.append({'params_defs': scenario_params + ie_adjust + lt5_vacc_adjust,
                                'hosp_reporting_frac': hrf,
                                'tags': {'escape_mult': escape_mult}})
            

In [15]:
# check how many scenarios there are
len(scenario_model_args)

3

In [16]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/11/04 20:51:06 INFO|============================================================
2022/11/04 20:51:06 INFO|============================================================
2022/11/04 20:51:06 INFO|============================================================
2022/11/04 20:51:09 INFO|-----------|{'escape_mult': 0.8} Retrieving vaccinations data
2022/11/04 20:51:09 INFO|-----------|{'escape_mult': 1.2} Retrieving vaccinations data
2022/11/04 20:51:09 INFO|-----------|{'escape_mult': 1} Retrieving vaccinations data
2022/11/04 20:51:11 INFO|-----------|{'escape_mult': 1.2} Constructing vaccination projections
2022/11/04 20:51:11 INFO|-----------|{'escape_mult': 1} Constructing vaccination projections
2022/11/04 20:51:11 INFO|-----------|{'escape_mult': 0.8} Constructing vaccination projections
2022/11/04 20:51:13 INFO|-----------|{'escape_mult': 1.2} Retrieving hospitalizations data
2022/11/04 20:51:13 INFO|-----------|{'escape_mult': 1} Retrieving hospitalizations data
2022/11/04 20:51:13 

2022/11/04 20:54:16 INFO|--------|{'escape_mult': 1.2, 'fit_batch': '4'}: Transmission control fit 5/7 completed in 12.997951041000022 seconds: {879: {'co': 0.8537621008229487}, 893: {'co': 0.8548151059725863}, 907: {'co': 0.8602703864039166}, 921: {'co': 0.8641581113624136}, 935: {'co': 0.8436859141450983}, 949: {'co': 0.8601838411050892}}
2022/11/04 20:54:16 INFO|---------|{'escape_mult': 1.2, 'fit_batch': '4'}: Running forward sim
2022/11/04 20:54:28 INFO|--------|{'escape_mult': 1, 'fit_batch': '5'}: Transmission control fit 6/7 completed in 13.328863124999998 seconds: {907: {'co': 0.8602810034542904}, 921: {'co': 0.8641772551596635}, 935: {'co': 0.8435794010139486}, 949: {'co': 0.8598380919697802}, 963: {'co': 0.8319369177431087}, 977: {'co': 0.8455663912654833}}
2022/11/04 20:54:28 INFO|---------|{'escape_mult': 1, 'fit_batch': '5'}: Running forward sim
2022/11/04 20:54:31 INFO|--------|{'escape_mult': 0.8, 'fit_batch': '5'}: Transmission control fit 6/7 completed in 13.717554499

2022/11/04 20:55:16 INFO|-----------------------|spec_ids: 4242,4243,4244


### Run the report for each fit model

In [17]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'scrabble'], from_date='2022-01-01')

2022/11/04 20:56:46 INFO|============================================================
2022/11/04 20:56:46 INFO|--------|Solving model
2022/11/04 20:56:47 INFO|============================================================
2022/11/04 20:56:47 INFO|--------|Solving model
2022/11/04 20:56:48 INFO|============================================================
2022/11/04 20:56:48 INFO|--------|Solving model
2022/11/04 20:56:51 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/04 20:56:52 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/04 20:56:52 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [18]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/11/04 20:59:06 INFO|Projecting
2022/11/04 20:59:06 INFO|
2022/11/04 20:59:28 INFO|{'escape_mult': 0.8, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/04 20:59:30 INFO|
2022/11/04 20:59:52 INFO|{'escape_mult': 1, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/04 20:59:54 INFO|
2022/11/04 21:00:17 INFO|{'escape_mult': 1.2, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/04 21:00:19 INFO|Running reports


In [ ]:
model.solution_ydf.to_csv('covid_model/output/20221109_scenario_projections.ipynb/state_model_solution_ydf.csv', index=False)